# 生物先端コース 学生実習 2025後期 3日目

## RNA-seq・発現解析・課題レポート

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/rnaseq.png?raw=true" alt="central_dogma" height="280px" align="middle">

本実習では、Genomics, Transcriptomics, Proteomicsに関する基本的なゲノム解析技術を学習していきます。

3日目は、RNA-seqによるTranscriptome解析について扱います。

遺伝子がどのようなときに発現するのかを調べる手法です。


# RNA-seq

昨日までに学んできた技術は、ある生物のDNA配列をシーケンスすることで、

生物のゲノム配列を解読したり、集団内での配列の違いを遺伝的変異としてショートリードを用いて検出することが出来る、という話でした。

ここからはRNAを対象としたシーケンス技術RNA-seqについて学んでいきます。

遺伝子は、mRNAに転写された後、アミノ酸配列に翻訳され、アミノ酸配列が立体構造を取りタンパク質となることで何らかの機能が働きます。(セントラルドグマ)

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/central_dogma.png?raw=true" alt="central_dogma" height="280px" align="middle">

DNA配列をロングリードやショートリード読み取るのがこれまで扱ってきたDNAシーケンスでしたが、RNAシーケンス(RNA-seq)では、転写されたmRNAを読み取ります。

RNA-seqによって、あるサンプルにおいて、どの遺伝子がどれだけ転写されているのか(**遺伝子の発現量**)、という情報を得ることが出来ます。

例えば、ある処理を行ったサンプルと行わなかったサンプルのRNA-seqを比較することで、処理によってどんな遺伝子が発現したのかが分かったりします。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/RNAseq_compare.png?raw=true" alt="RNAseq_compare" height="350px" align="middle">

# RNA-seqのマッピング

RNA-seqについても、遺伝的変異の検出を行った時と同じようにまずはマッピングを行います。

転写されている遺伝子のmRNAを抽出し、ショートリードとしてシーケンスを行います。

当然どこ由来の配列データか不明なので、マッピングにより、どのゲノム領域が転写されたものかを解析します。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/mRNA_mapping.png?raw=true" alt="central_dogma" height="350px" align="middle">

ただし、RNA-seqの場合、昨日のDNA配列をマッピングするのとは異なる点があります。

生物学の復習になりますが、mRNAに転写される際には、余分なもの(intron)を切り離してexonのみを再度つなぎ合わされるスプライシングが行われます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/splicing.png?raw=true" alt="splicing" height="250px" align="middle">

RNA-seqでは、転写されたmRNAをシーケンスするので、このexon領域**のみ**をシーケンスしていることになります。

そのため、DNAシーケンスではリファレンスゲノムの配列と一致する場所をそのまま探して貼り付ければ良かったのに対して、RNA-seqで得られるリードは**分割して飛び飛びにある複数のexon領域にまたがって張り付ける**ということをする必要があります。

つまり、DNA-seqと異なり、RNA-seqのリードをリファレンス配列にマッピングする際には、intronのギャップを考慮してマッピングする必要があります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/RNA_mapping.png?raw=true" alt="RNA_mapping" height="200px" align="middle">

この様な状況に対応するため、DNA配列のマッピングには`bwa`というプログラムを使用しましたが、RNA-seqのマッピングにはそれ用のプログラムを使用する必要があります。

[Hisat2](http://daehwankimlab.github.io/hisat2/)や[STAR](https://github.com/alexdobin/STAR)などが、RNA-seqに適した(ギャップを考慮してくれる)マッピングツールとなります。

今回は[Hisat2](http://daehwankimlab.github.io/hisat2/)を使用しましょう。

ターミナルを開き、下記コマンドでHisat2をインストールしておきましょう。

```
apt install hisat2
```

## 構築したリファレンス配列にRNA-seqリードのマッピング

それでは、初日リファレンスゲノムを作成したいもち病菌(菌株GZ)を用いたRNA-seq解析を実際に実施していきましょう。


## シーケンス結果の取得

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/mRNA_sequencing.png?raw=true" alt="central_dogma" height="250px" align="middle">

まずはデータセットのダウンロードをしていきます。

初日に実行したアセンブルの結果であるゲノム配列は

* https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/assembly.fasta

いもち病菌のRNAを次世代シーケンサーで読んだRNA-seqの配列データは

* https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_0h_rep1.fastq.gz

にあります。

今回は前回のペアエンドではなくシングルエンドのショートリードになります。

(今回は別株のデータを仮で使用しています。)

左下からターミナルを開いて`wget`コマンドで上の2つのデータをダウンロードしてみてください。

```
wget ダウンロードしたいファイルのURL
```

また、配列を扱うツール`seqkit`もインストールしておきます。

```
apt install seqkit
```

`less`や`seqkit`コマンドを使用して見てみると、今回扱うRNA-seqのショートリードは昨日扱ったDNAのリードとはあまり違いはありません。

ただDNA由来か、RNA由来か、というだけです。

## 配列データのマッピング

では続いて、ショートリードとして読んだmRNAの配列をリファレンスゲノムに貼り付けていきます。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/mRNA_alignment.png?raw=true" alt="central_dogma" height="250px" align="middle">

先ほども説明した通り、今回はRNA-seqのマッピングに適した`Hisat`というプログラムを使用します。


### リファレンスゲノムのindex作成

前回同様、まずマッピングする前にリファレンスゲノムのindexファイルというものを作成します。

indexファイルとは、**索引ファイル**のことを指しましたが、この索引ファイルは基本的にツール毎に作成する必要があります。

`Hisat2`のためのindexファイルは下記コマンドで作成出来ます。

```
hisat2-build リファレンスゲノムのFASTAファイル名 assembly
```

FASTAファイル名を正しく入力して上記コマンドを実行し、

`assembly.数字.ht2`というファイルがたくさん作成されていれば成功です。


### マッピングの開始

indexファイルを作成したら、実際にRNA-seqリードのマッピングをしていきます。

`hisat2`コマンドでマッピングができます。

```
hisat2 -x 作成したindex名 -U ショートリードのファイル名 -S 出力ファイル名
```

今回は作成したindex名が`genome_index`、それぞれのショートリードが`infect_0h_rep1.fastq.gz`になるので、

```
hisat2 -x genome_index -U infect_0h_rep1.fastq.gz -S infect_0h_rep1.sam
```
というコマンドになります。出力ファイル名は`infect_0h_rep1.sam`としています。

(そんなに時間はかからないと思います。)

プログラムが走り終えると、`infect_0h_rep1.sam`というファイルが出来ているかと思います。

SAMファイルはマッピング結果が記されていましたが、今回は使用したRNA-seqのマッピング結果の情報がテキストとして保存されています。

前回同様、SAMファイルはファイルサイズが大きくなってしまうので、BAMファイルに変換し、位置もソートしていきます。

SAMファイルを操作するためのプログラム`samtools`を使用して整理します。

下記コマンドで`samtools`をインストールします。

```
apt install samtools
```

samtoolsがインストール出来たら、`samtools sort`というコマンドに、オプションで`-O bam`とつけることで、SAM形式からBAM形式に変換・位置をソートできました。

```
samtools sort -O bam SAMファイル名 > BAMファイル名
samtools index BAMファイル名
```
今回のSAMファイルに対しても昨日と同じように動かして`infect_0h_rep1.bam`というBAMファイルを作成してみてください。

DNAの解析では集団を扱うと大量のファイルが出来てサイズが大きくなる可能性がありましたが、

RNA-seqの場合、集団の解析というよりは様々なコンディション(処理A, 処理B, 温度C, 温度D, ... etc)毎にRNA-seqを実施し比較することが多いので、そういった意味でサイズが非常に大きくなります。

よってRNA-seqでも基本的にBAMファイルとしてマッピング結果は保存することが多いです。


### マッピング結果の可視化

これでBAM(SAM)ファイルとして、大量のRNA-seqリードをマッピングした結果が得られました。

昨日通り、`IGV`を使用したマッピング結果の可視化を行います。

下のコマンドを動かすと、Google Colab上でIGVが使用可能になります。

```
pip install -q igv-notebook==0.3.1
wget -q -O modules.py https://raw.githubusercontent.com/slt666666/FAO_lecture/main/FAO_2024/data/modules.py
wget -q https://github.com/CropEvol/lecture/raw/master/textbook_2022/scripts/igv_prep.py -O igv_prep.py

samtools faidx assembly.fasta
```

プログラムのダウンロードが終わったら、下のプログラムを動かすことでIGVが動きます。

In [ ]:
## Visualize alignment results
## import libraries
import igv_notebook
from modules import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
# show reference genome
ref = RefTrack({ "fastaPath":"assembly.fasta", "indexPath":"assembly.fasta.fai", "id":"Strain_X" })
# show aligned reads
B = BamTrack({ "name":"infect_0h_rep1", "path":"infect_0h_rep1.bam", "indexPath":"infect_0h_rep1.bam.bai", "viewAsPairs":True })
# visualize
b = igv_notebook.Browser(ref)
b.load_track(B)


可視化してみると、どのあたりのゲノム領域が盛んに転写されているのか、ゲノム上でどこからどこまでがexon領域で、どこがintron領域か、ということが分かります。

また、exon領域にまたがるリードがいくつかあることが分かるかと思います。このことから、どのexonが結合して1つのmRNAとなっているか(どのexonが1つの遺伝子から来ているか)も調べることが出来ます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/gene_annotation_RNAseq.png?raw=true" alt="gene_annotation_RNAseq" height="350px" align="middle">

## 遺伝子のアノテーション

さすがに目視で全てのexon領域をチェックすることは難しいので、マッピング結果のBAMファイルから遺伝子のexon領域・intron領域の情報を取り出すプログラムを使用します。

この様な、どの領域に遺伝子が位置し、どの部分がexonでどの部分がintronかを検出することを**アノテーション**と呼びます。

RNA-seqの結果からアノテーションを実施する事が出来る有名なプログラムとしては[stringtie](https://ccb.jhu.edu/software/stringtie/)や[PsiCLASS](https://github.com/splicebox/PsiCLASS)等があります。

ロングリードのRNA-seqに対応していたり、スプライシングバリアントの検出が得意だったり、プログラム毎に特徴があります。

今回は`stringtie`を使用するので、今までと同じようにインストールしましょう。

```
apt install stringtie
```

`stringtie`を使用することで、RNA-seqのBAMファイルからアノテーションを実施する事が出来ます。

アノテーションを実施するとexonの位置などをリスト化したGTFファイルというものが作成されます。

GTFとはGene Feature Formatを意味します。

コマンドはシンプルで、使用するBAMファイルと作成するGTFファイル名を引数で指定するだけです。

```
stringtie BAMファイル名 -o 出力するGTFファイル名
```

今回は`transcripts.gtf`という名前のファイルに出力してみましょう。

上手く動かす事が出来れば、`transcripts.gtf`が作成されます。

このファイルの中身を`less -S`コマンドで確認すると

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/stringtie_results.png?raw=true" alt="central_dogma" height="400px" align="middle">


この様な形で、どの染色体、どの位置がexonか、等の情報が記されています。

IGVを使用すると、先ほどのマッピング結果に遺伝子のアノテーション情報を追加する事が出来ます。

In [ ]:
## RNAseqのアライメント結果を見る
## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"assembly.fasta", "indexPath":"assembly.fasta.fai", "id":"Strain_X" })
## 準備: Reads info
A = BamTrack({ "name":"infect_0h_rep1", "path":"infect_0h_rep1.bam", "indexPath":"infect_0h_rep1.bam.bai", "viewAsPairs":True })
## IGV表示
b = igv_notebook.Browser(ref)
## gff 表示
b.load_track({"name": "Annotations", "type": "annotation", "format": "gtf", "displayMode": "EXPANDED", "path": "transcripts.gtf"})
b.load_track(A)

### そのほかのアノテーションツール・必要なデータについて

遺伝子のアノテーションには、RNA-seqを使用せず、機械学習モデルや他の生物種の遺伝子アノテーション情報から予測を行う手法も開発されています。

例えば[Augustus](https://bioinf.uni-greifswald.de/augustus/)や[Helixer](https://github.com/weberlab-hhu/Helixer?tab=readme-ov-file)など。

ただし、あくまでも予測の域を出ないので、実際に**動いている遺伝子**を正しくアノテーションするにはRNA-seqが必須になります。

また、例えば植物だと葉でのみ発現している遺伝子もあれば、根でしか発現していない遺伝子、熱ストレス下でのみ発現する遺伝子などが存在します。

その様な遺伝子をアノテーションするためには、葉のサンプルから得られたRNA-seqのみを使用するのではなく、様々な部位・コンディションのサンプルを用いてRNA-seqを行い、遺伝子のアノテーションをする必要があります。

## 発現量のカウント

DNA配列のマッピングでは遺伝的な変異を検出しましたが、RNA-seqではどの遺伝子がどれだけ発現している(転写されている)のか、という情報を得ることが目的になります。

※RNA-seqを使用して遺伝的変異を検出することも可能です。


RNA-seqで読まれるリードは、抽出したmRNAに依存します。

大量に転写されている遺伝子のmRNAはその分よくシーケンスされていますし、あまり発現していない遺伝子のmRNAはあまりシーケンスされていません。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/read_count.png?raw=true" alt="central_dogma" height="250px" align="middle">

よって、各遺伝子(exon)毎に、大量のリードが張り付いているものはよく転写しており、あまりリードが張り付いていない領域からは転写量が少ない、ということが分かります。

そのため、この張り付いたリードの量をカウントすることで各遺伝子の発現量を定量化する事が出来ます。

リード量をカウントするプログラムとしては`featureCounts`等が有名です。

`featureCounts`は`subread`というツールセットの中のプログラムの1つなので、下記コマンドでインストールできます。

```
apt install subread
```

`featureCounts`を使用して、RNA-seqをマッピングしたBAMファイルから遺伝子毎の発現量(リードカウント)を得るには、下記のコマンドを使用します。

```
featureCounts -a GTFファイル -o 出力ファイル名 BAMファイル
```

今回は出力ファイル名を`counts.txt`として、`featureCounts`を動かしてみましょう。

`GTFファイル`は先ほど作成した`transcripts.gtf`、`BAMファイル`は`infect_0h_rep1.bam`になります。

得られたcounts.txtというファイルを見ると、各遺伝子毎にどれだけの数のリードが張り付いたか見る事が出来ます。

どの遺伝子の発現量が多いか少ないか、定量的に確認可能です。




### 複数サンプルの遺伝子発現データ

ここまで、遺伝子の発現量を確認する一般的な方法を扱いましたが、

通常の研究では複数サンプルの遺伝子発現量を比較し解析する場合が多いです。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/compare_RNAseq.png?raw=true" alt="central_dogma" height="250px" align="middle">

そこで、次に複数サンプルの遺伝子発現量の扱いを見てみます。

先ほどは通常時のいもち病菌のRNA-seqのデータを使用しました。

追加サンプルとして、感受性を示すオオムギ品種Nigrateにいもち病菌を接種し、48時間経った時のいもち病菌サンプルのRNA-seqデータを扱います。

通常時の遺伝子発現量と、感染時の遺伝子発現量を比較することで、どんな遺伝子が感染に必要か確認する事が出来ます。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/mRNA_compare.png?raw=true" alt="central_dogma" height="250px" align="middle">

感染時のRNA-seqデータは、

* https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_48h_rep1.fastq.gz

にあります

`wget`コマンドでデータをダウンロードし、このRNA-seqのショートリード(infect_48h_rep1.fastq.gz)を使用して、

さきほどと同じ過程でマッピング結果の`BAMファイル`を作成してみてください。

1. マッピング
```
hisat2 -x 作成したindex名 -U ショートリードのファイル名 -S 出力ファイル名
```

2. マッピング結果のソート、BAM変換
```
samtools sort -O bam SAMファイル名 > ファイル名.bam
samtools index BAMファイル名
```

`infect_48h_rep1.bam`と`infect_48h_rep1.bam.bai`いうファイルが出来ていれば成功です。


続いて、通常時の遺伝子発現量と感染時の遺伝子発現量を比較します。

`featureCounts`では複数のBAMファイルを扱うことが可能です。

```
featureCounts -a GTFファイル -o 出力ファイル名 BAMファイル1 BAMファイル2 BAMファイル3...
```

最後に指定するBAMファイルを`infect_0h_rep1.bam`と`infect_48h_rep1.bam`の2つ指定して動かしてみてください。

`-o`で指定する出力Fファイル名は`counts.txt`としておきます。

このファイルを`less -S`コマンドなどで見てみると、

`infect_0h_rep1`でのリードカウント、`infect_48h_rep1`のリードカウントが表示されています。

このカウントを比較していくわけですが、RNA-seqでは張り付いたリードの数を単純に比較するだけでは、サンプル間の比較はできません。

まず、リードカウントは補正する必要があります。[詳しくはゲノム解析入門のRNA-seqの章](https://colab.research.google.com/github/CropEvol/lecture/blob/master/textbook_2024/RNAseq_1.ipynb#scrollTo=Mwy80FwugqxR)を見てください。

<br>

また、基本的にRNA-seqは同じ条件で反復を少なくとも３回とることが多いです。

遺伝子の発現量はブレが大きいので、反復をとってより信頼性を高めた結果を得る必要があります。

例えば今回の様に非感染時と感染時の遺伝子発現量を比較したい場合は、

非感染時のRNA-seqを３回、感染時のRNA-seqを３回行い、総合的に比較する形になります。

* 非感染時の３反復
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_0h_rep1.fastq.gz
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_0h_rep2.fastq.gz
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_0h_rep3.fastq.gz

* 感染時の３反復
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_48h_rep1.fastq.gz
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_48h_rep2.fastq.gz
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_48h_rep3.fastq.gz

これら６サンプルのRNA-seqの結果を使用します。

先ほどやったのと同様の解析を６サンプル使用して行います。

ただ、同じ作業を繰り返すことになるので、６回の解析を一気に実行するシェルスクリプトを用意したので下記コマンドを実行してください。

```
wget https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/RNA_seq.sh

bash RNA_seq.sh
```

`counts_0h_vs_48h.txt`というファイルが出来ていればOKです。

さっきまでやっていたリードカウントの算出を６サンプル全てに対して行った形になります。

ついでに遺伝子のアノテーション情報も全サンプルを使用したものにしてあります。

この反復を含むリードカウントのデータを使用してサンプル間の比較などを行っていきます。

## DEG解析

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/DEG.png?raw=true" alt="central_dogma" height="400px" align="middle">

サンプル間で発現量が大きく異なっている遺伝子を検出する解析を**DEG**(Differentially expressed gene)解析と呼びます。

DEG解析は自分でコツコツプログラムを書いても出来る解析ですが、

Rの[edgeR](https://bioconductor.org/packages/release/bioc/vignettes/edgeR/inst/doc/edgeRUsersGuide.pdf)や[DEseq2](https://bioconductor.org/packages/release/bioc/manuals/DESeq2/man/DESeq2.pdf)といったパッケージを活用することで、

リードカウントの補正も含んだ、サンプル間で遺伝子発現量の比較を簡単に行う事が出来ます。

今回は`DEseq`をPythonで動かす事が出来るプログラム[PyDESeq](https://pydeseq2.readthedocs.io)を使用してDEG解析を行います。

下記コマンドで`PyDESeq`をインストールします。

```
pip install pydeseq2
```

細かい説明は省きますが[公式のチュートリアル](https://pydeseq2.readthedocs.io/en/stable/auto_examples/index.html)を読んでいけば必要なコードやコマンドが全て記載されています。

下記のコードを動かすことで感染48h後と非感染時(0h)の発現変動遺伝子(DEG)検出が実施されます。


In [ ]:
import pandas as pd
from pydeseq2.utils import load_example_data
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

# データの整備
counts_df = pd.read_csv("counts_0h_vs_48h.txt", sep="\t", comment="#").set_index("Geneid").iloc[:, 5:].T
metadata = pd.DataFrame(index=counts_df.index, data={"condition":["0h", "0h", "0h", "48h", "48h", "48h"]})

# 発現量の極端に少ない遺伝子を除去
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
counts_df = counts_df[genes_to_keep]

# リードカウントの補正&検定など
inference = DefaultInference()
dds = DeseqDataSet(
  counts=counts_df,
  metadata=metadata,
  design_factors="condition",
  refit_cooks=True,
  inference=inference,
)
dds.deseq2()
stat_res = DeseqStats(dds, contrast=["condition", "48h", "0h"], inference=inference)
stat_res.summary()

下記コマンドで解析結果を保存する事が出来、結果はExcel等で開く事が出来ます。

In [ ]:
# 解析結果の保存
stat_res.results_df.to_csv("DEG_0h_vs_48h.csv")

得られた結果の`log2FoldChange`が比較してどのくらい発現量に差があったかを示しています。

"log2FoldChange"が1だと2倍違い、2だと4倍違ってくるという形です。

`padj`が補正されたp値(adjusted-pvalue)を示しています。

全ての遺伝子について載っているので、"発現量に差がある"という基準のものを抽出する必要があります。

`log2FoldChange`の絶対値が1以上かつ補正されたp値`padj`が0.05未満の遺伝子をDEGとする場合が多いです。

In [ ]:
# 発現量の変動が強い上位10遺伝子を表示
stat_res.results_df.sort_values(by=["padj"]).head(10)

IGVでリードの張り付き具合を見てみると差がよく分かります。

In [ ]:
## RNAseqのアライメント結果を見る
## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"assembly.fasta", "indexPath":"assembly.fasta.fai", "id":"Strain_X" })
## 準備: Reads info
A = BamTrack({ "name":"infect_0h_rep1", "path":"infect_0h_rep1.bam", "indexPath":"infect_0h_rep1.bam.bai", "viewAsPairs":True })
B = BamTrack({ "name":"infect_48h_rep1", "path":"infect_48h_rep1.bam", "indexPath":"infect_48h_rep1.bam.bai", "viewAsPairs":True })
## IGV表示
b = igv_notebook.Browser(ref)
## gff 表示
b.load_track({"name": "Annotations", "type": "annotation", "format": "gtf", "displayMode": "EXPANDED", "path": "transcripts.gtf"})
b.load_track(A)
b.load_track(B)

### BLASTで発現変動の大きかった遺伝子を調べてみる

こうしてみると、`STRG.1697`という遺伝子は感染時に発現量がガクッと下がっています。

逆に`STRG.1269`という遺伝子は感染時に発現量が爆発的に上がっています。

これらの遺伝子の機能が気になるので、まずは各遺伝子の配列を取りに行きます。

`gffread`というツールを使うと、先ほど作成した遺伝子アノテーション情報のGTFファイルとゲノム配列から遺伝子毎の配列を取り出す事が出来ます。

```
apt install gffread
```

下記コマンドで各exon領域の配列を全て取り出せます。

```
gffread transcripts.gtf -g assembly.fasta -w transcripts.fasta
```

このコマンドで作成された`transcripts.fasta`に全遺伝子の配列が入っています。

さて、ここから目的の`STRG.1697`や`STRG.1269`を探すわけですが、今度は`seqkit`を使用した下記コマンドで抽出可能です。

※この時、探したい遺伝子名の後ろに".1"をつけてください。

例) `"STRG.1697"`を探す場合は`"STRG.1697.1"`

```
seqkit grep -p 探したい遺伝子名 transcripts.fasta > 出力ファイル名
```

これで遺伝子名をキーに配列を出力する事が出来ます。

出力された配列を[BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi)にかけて、発現変動が大きい遺伝子がどのような機能を持っていそうか調べてみる事が出来ます。




### DEG解析の可視化: Volcano plot

この様なDEG解析の結果はVolcano plotの形で可視化されることが多いです。

今回はPythonを使用している都合上プログラムを書いていますが、Rには自動的にVolcano plotを描いてくれるパッケージなどもあります。

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df = stat_res.results_df
df['nlog10'] = -np.log10(df.padj)

def dot_color(a):
    logFC, nlog10 = a

    if abs(logFC) <= 1 or nlog10 <= 2:
        return 'Not significant'
    if logFC > 1 and nlog10 >2:
        return 'Up'
    if logFC < -1 and nlog10 >2:
        return 'Down'
df['color'] = df[['log2FoldChange', 'nlog10']].apply(dot_color, axis = 1)

plt.figure(figsize = (7,7))

ax = sns.scatterplot(data = df, x = 'log2FoldChange', y = 'nlog10',
                    hue = 'color', hue_order = ['Not significant', 'Up', 'Down'],
                    palette = ['dimgrey', 'firebrick', 'darkslateblue'])

ax.axhline(2, zorder = 0, c = 'k', lw = 2, ls = '--')
ax.axvline(1, zorder = 0, c = 'k', lw = 2, ls = '--')
ax.axvline(-1, zorder = 0, c = 'k', lw = 2, ls = '--')

for axis in ['bottom', 'left']:
    ax.spines[axis].set_linewidth(2)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.tick_params(width = 2)

plt.xticks(size = 12, weight = 'bold')
plt.yticks(size = 12, weight = 'bold')

plt.xlabel("$log_{2}$ fold change", size = 15)
plt.ylabel("-$log_{10}$ q-value", size = 15)

plt.show()

# レポート課題

今回は扱った解析が多岐にわたるので、全体のレポートを書いてもらうのは大変だと思います。そこでRNA-seqの解析に絞ったレポートを書いて下さい。

[レポート課題のページ](https://colab.research.google.com/github/CropEvol/lecture/blob/master/exp_2025_Oct/RNA_seq_report.ipynb)



明日は遺伝子が転写翻訳された後の、タンパク質の構造予測を行います。

